# Simple Index Demo + ChatGPT

Use a very simple wrapper around the ChatGPT API

#### Load documents, build the GPTSimpleVectorIndex

In [10]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from gpt_index import GPTSimpleVectorIndex, SimpleDirectoryReader, LLMPredictor
from langchain.llms import OpenAIChat
from IPython.display import Markdown, display

In [11]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [12]:
index = GPTSimpleVectorIndex(documents, chunk_size_limit=512)

INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
> [build_index_from_documents] Total LLM token usage: 0 tokens
> [build_index_from_documents] Total LLM token usage: 0 tokens
> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 18579 tokens
> [build_index_from_documents] Total embedding token usage: 18579 tokens
> [build_index_from_documents] Total embedding token usage: 18579 tokens
> [build_index_from_documents] Total embedding token usage: 18579 tokens


In [13]:
# LLM Predictor (gpt-3.5-turbo)
llm_predictor = LLMPredictor(llm=OpenAIChat(temperature=0, model_name="gpt-3.5-turbo"))

#### Query Index

In [14]:
# set Logging to DEBUG for more detailed outputs
response = index.query(
    "What did the author do growing up?", 
    llm_predictor=llm_predictor,
    similarity_top_k=3
)

INFO:root:> [query] Total LLM token usage: 2044 tokens
> [query] Total LLM token usage: 2044 tokens
> [query] Total LLM token usage: 2044 tokens
> [query] Total LLM token usage: 2044 tokens
INFO:root:> [query] Total embedding token usage: 8 tokens
> [query] Total embedding token usage: 8 tokens
> [query] Total embedding token usage: 8 tokens
> [query] Total embedding token usage: 8 tokens


In [15]:
display(Markdown(f"<b>{response}</b>"))

<b>

The author worked on writing and programming before college, including writing short stories and trying programming on an IBM 1401 in 9th grade using an early version of Fortran. They continued to write essays throughout 2020, but in late 2015, they spent three months solely working on programming their interpreter, Bel. They worked on Bel so intensively that they had a decent chunk of the code in their head at any given time and could write more there. They even figured out how to deal with some problem involving continuations while watching their kids play in the tide pools. The author moved to England in the summer of 2016 and continued to work on Bel until it was finally finished in the fall of 2019. After finishing Bel, the author returned to writing essays and reflecting on what to work on next.</b>

In [ ]:
# NOTE: this may not give you a satisfactory answer! 
response = index.query(
    "What did the author do during his time at RISD?", 
    llm_predictor=llm_predictor,
    similarity_top_k=5
)

In [42]:
display(Markdown(f"<b>{response}</b>"))

<b>The original answer remains relevant and does not need to be refined as the new context does not provide any information about what the author did during his time at RISD.</b>

#### Query Index (Refine Prompt Tailored to ChatGPT)

We use a custom Refine Prompt tailored to the ChatGPT API.
It effectively breaks the refine prompt into separate "conversational" messages.

In [43]:
from gpt_index.prompts.chat_prompts import CHAT_REFINE_PROMPT

In [ ]:
response = index.query(
    "What did the author do during his time at RISD?", 
    llm_predictor=llm_predictor,
    refine_template=CHAT_REFINE_PROMPT,
    similarity_top_k=5
)

In [45]:
display(Markdown(f"<b>{response}</b>"))

<b>During his time at RISD, the author took art classes at Harvard while also being in a PhD program in computer science. He also painted and lived in New York, where he became a de facto studio assistant for a painter named Idelle Weber. He wrote a book on Lisp to earn money and imagined himself living frugally off the royalties and spending all his time painting. He also became interested in the World Wide Web and started a company to put art galleries online. However, he dropped out of RISD in 1993 and moved to New York to pursue his career as an artist. He lived in a rent-controlled apartment in Yorkville and taught himself to paint while freelancing as a Lisp hacker to make ends meet.</b>

### [Beta] Use ChatGPTLLMPredictor

Very simple GPT-Index-native ChatGPT wrapper. Note: this is a beta feature. If this doesn't work please
use the suggested flow above.

In [3]:
# use ChatGPT [beta]
from gpt_index.langchain_helpers.chatgpt import ChatGPTLLMPredictor

llm_predictor = ChatGPTLLMPredictor()

In [ ]:
response = index.query(
    "What did the author do during his time at RISD?", 
    llm_predictor=llm_predictor
)